In [1]:
import networkx as nx
import numpy as np
import torch
import pickle
from torch_geometric.utils import from_networkx

In [2]:
### expect dictionary dataset
with open("data/ltspice_demos_LP_complete.pkl",'rb') as f:
    dataset = pickle.load(f)
    f.close()
with open('data/ltspice_demos_label_mapping.pkl', 'rb') as f:
    mapping = pickle.load(f)
    f.close()
train_x = dataset['train_x']
test_x = dataset['test_x'] 
# train_x.extend(test_x)

In [3]:
len(train_x), len(test_x)

(116, 35)

In [4]:
dataset.keys()

dict_keys(['train_x', 'test_x'])

In [5]:

X,all_graphs = [],[]
max_ = 0
for g in train_x:
    if g.number_of_nodes()>max_:
        max_ = g.number_of_nodes()
    for i, n in enumerate(g.nodes()):
        feat = np.zeros((5,),dtype = np.float64)
        node_type = g.nodes[n]['type']
        index = mapping.get(node_type)
        feat[index] = 1.0
        X.append(feat)
    g_ = nx.Graph()
    g_.add_nodes_from(g.nodes())
    g_.add_edges_from(g.edges())
    all_graphs.append(g_)
graph = nx.disjoint_union_all(all_graphs)
print("total number of nodes and edges, max_number_of_nodes:", graph.number_of_nodes(), graph.number_of_edges(),max_)
data = from_networkx(graph,group_node_attrs=None,group_edge_attrs=None )
data.x = torch.from_numpy(np.array(X)).float()
torch.save(data,'data/ltspice_demos_torch_LP_complete.pt')

total number of nodes and edges, max_number_of_nodes: 2455 3430 46
